In [ ]:
import pandas as pd
import csv
import plotly.express as px


import numpy as np
from scipy.optimize import curve_fit
import pwlf

In [ ]:
# Importing files (just once because it takes LOOOOOOOOONG)

# Results (not that useful)
results_2021_import = pd.read_excel('/Users/nicolas/Documents/GitHub/nseemann-msc-BESS/dev_OOP/20_data/regelleistung/RESULT_LIST_ANONYM/RESULT_LIST_ANONYM_ENERGY_MARKET_aFRR_2021-01-01_2021-12-31.xlsx')

# Only goes up to 2024-06-11
#bids_june_2024_import = pd.read_excel('../../20_data/regelleistung/RESULT_LIST_ANONYM_ENERGY_MARKET_aFRR_2024-06-01_2024-06-30.xlsx')
#bids_june_2024_import.to_csv('../../20_data/regelleistung/RESULT_LIST_ANONYM_ENERGY_MARKET_aFRR_2024-06-01_2024-06-30.csv')
# bids_june_2024_import = pd.read_csv('../../20_data/regelleistung/RESULT_LIST_ANONYM_ENERGY_MARKET_aFRR_2024-06-01_2024-06-30.csv',low_memory=False)
# bids_june_2024_import.drop(columns=['Unnamed: 0'], inplace=True)
# bids_june_2024_import
# Whole month
# activated_all_time_import = pd.read_csv('../../20_data/netztransparenz/Aktivierte aFRR qualitaetsgesichert [2024-09-26 15-50-49].csv', delimiter = ';')

/Users/nicolas/miniforge3/envs/analysis/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:241: UserWarning:

Workbook contains no default style, apply openpyxl's default



In [50]:
results_2021_import.PRODUCT.unique()

array(['NEG_00_04', 'NEG_04_08', 'NEG_08_12', 'NEG_12_16', 'NEG_16_20',
       'NEG_20_24', 'POS_00_04', 'POS_04_08', 'POS_08_12', 'POS_12_16',
       'POS_16_20', 'POS_20_24'], dtype=object)

In [51]:
# Preparing bidding data

results_2022 = results_2022_import.copy()
results_2022['DELIVERY_DATE'] = pd.to_datetime(results_2022.DELIVERY_DATE,format = "%Y.%m.%d")

# Translate payment direction into sign for price
results_2022['ENERGY_PRICE_[EUR/MWh]'] = results_2022['ENERGY_PRICE_[EUR/MWh]']*(((results_2022['ENERGY_PRICE_PAYMENT_DIRECTION'] == 'PROVIDER_TO_GRID') * -1) + ((results_2022['ENERGY_PRICE_PAYMENT_DIRECTION'] == 'GRID_TO_PROVIDER') * 1))

# Sort by Dalivery_date, product then energy price (check order)
bids_sorted = results_2022.sort_values(['DELIVERY_DATE','PRODUCT','ENERGY_PRICE_[EUR/MWh]'])

# Cumulative sums of offered capacity
bids_sorted['sum_cap'] = bids_sorted.groupby(['DELIVERY_DATE','PRODUCT'])['OFFERED_CAPACITY_[MW]'].cumsum()

bids_sorted = bids_sorted.set_index(['DELIVERY_DATE','PRODUCT'])

bids_sorted

TYPE_OF_RESERVES  ENERGY_PRICE_[EUR/MWh]  \
DELIVERY_DATE PRODUCT                                              
2021-01-01    NEG_00_04             aFRR                   -8.34   
              NEG_00_04             aFRR                   -8.04   
              NEG_00_04             aFRR                   -7.89   
              NEG_00_04             aFRR                   -7.85   
              NEG_00_04             aFRR                   -7.83   
...                                  ...                     ...   
2021-09-02    POS_00_04             aFRR                 1999.00   
              POS_00_04             aFRR                 2420.00   
              POS_00_04             aFRR                 2555.00   
              POS_00_04             aFRR                 6999.00   
              POS_00_04             aFRR                 9999.00   

                        ENERGY_PRICE_PAYMENT_DIRECTION  OFFERED_CAPACITY_[MW]  \
DELIVERY_DATE PRODUCT                                                           
2021-01-01    NEG_00_04               PROVIDER_TO_GRID                      5   
              NEG_00_04               PROVIDER_TO_GRID                      5   
              NEG_00_04               PROVIDER_TO_GRID                      5   
              NEG_00_04               PROVIDER_TO_GRID                      5   
              NEG_00_04               PROVIDER_TO_GRID                      5   
...                                                ...                    ...   
2021-09-02    POS_00_04               GRID_TO_PROVIDER                     14   
              POS_00_04               GRID_TO_PROVIDER                      8   
              POS_00_04               GRID_TO_PROVIDER                      5   
              POS_00_04               GRID_TO_PROVIDER                      7   
              POS_00_04               GRID_TO_PROVIDER                     17   

                         ALLOCATED_CAPACITY_[MW] COUNTRY NOTE  sum_cap  
DELIVERY_DATE PRODUCT                                                   
2021-01-01    NEG_00_04                        5      DE  NaN        5  
              NEG_00_04                        5      DE  NaN       10  
              NEG_00_04                        5      DE  NaN       15  
              NEG_00_04                        5      DE  NaN       20  
              NEG_00_04                        5      DE  NaN       25  
...                                          ...     ...  ...      ...  
2021-09-02    POS_00_04                       14      DE  NaN      578  
              POS_00_04                        8      DE  NaN      586  
              POS_00_04                        5      DE  NaN      591  
              POS_00_04                        7      DE  NaN      598  
              POS_00_04                        5      DE  NaN      615  

[1048575 rows x 8 columns]

In [74]:
first_D_pos_001['ENERGY_PRICE_[EUR/MWh]'].values

array([ 118.87,  119.  ,  125.  ,  125.91,  125.92,  125.93,  125.94,
        126.84,  128.  ,  130.34,  130.66,  130.66,  130.66,  131.  ,
        137.  ,  137.13,  137.74,  138.02,  139.  ,  141.37,  141.77,
        142.  ,  143.  ,  143.39,  143.77,  143.84,  144.  ,  144.01,
        144.8 ,  145.  ,  145.31,  145.68,  146.87,  147.2 ,  147.53,
        148.95,  149.  ,  149.37,  151.21,  153.  ,  153.05,  154.89,
        155.05,  156.74,  157.61,  158.58,  160.  ,  160.42,  161.  ,
        162.26,  162.5 ,  162.94,  164.1 ,  165.34,  165.6 ,  165.95,
        167.79,  168.17,  169.6 ,  169.63,  171.47,  172.39,  173.31,
        173.32,  174.39,  174.82,  175.16,  176.01,  177.  ,  177.55,
        178.22,  179.66,  180.  ,  184.42,  189.47,  190.  ,  190.37,
        191.28,  192.19,  193.11,  194.03,  194.95,  195.88,  196.82,
        197.75,  198.  ,  198.95,  199.99,  200.  ,  209.14,  210.74,
        213.18,  215.01,  216.45,  220.65,  225.02,  225.95,  225.96,
        225.97,  225

In [90]:
my_pwlf.slopes, my_pwlf.intercepts, my_pwlf.fit_breaks

np.diff(my_pwlf.fit_breaks)

array([2.64752222e+02, 1.29546205e+02, 1.54662821e+00, 1.01820176e+02,
       1.09457576e+01, 5.65261048e+00, 1.10436756e+01, 2.55533354e+01,
       2.14654163e-02, 3.61792412e+00])

In [ ]:


first_D_pos_001 = bids_sorted.loc['2021-08-29','POS_04_08']
first_D_pos_001['sum_cap'] = first_D_pos_001['sum_cap']/4
first_D_pos_001['avg_price'] = (first_D_pos_001['ENERGY_PRICE_[EUR/MWh]']*first_D_pos_001['OFFERED_CAPACITY_[MW]']/4).cumsum()/first_D_pos_001['sum_cap']

# Plotting the merit order using Plotly
fig = px.line(first_D_pos_001, x='sum_cap', y='ENERGY_PRICE_[EUR/MWh]', markers=True, title='Merit Order for POS_001 on 2024-06-01')
fig.update_layout(xaxis_title='Activated energy (15min)', yaxis_title='Energy Price [EUR/MWh]')
fig.add_scatter(x=first_D_pos_001['sum_cap'], y=first_D_pos_001['avg_price'], mode='lines', name='Average Price', line=dict(color='black', width=2))

# Fit the piecewise linear model
my_pwlf = pwlf.PiecewiseLinFit(first_D_pos_001['sum_cap'], first_D_pos_001['ENERGY_PRICE_[EUR/MWh]'])
res = my_pwlf.fit(10)

# Predict the values
x_hat = np.linspace(first_D_pos_001['sum_cap'].min(), first_D_pos_001['sum_cap'].max(), num=1000)
y_hat = my_pwlf.predict(x_hat)

# Plot the piecewise linear fit
fig.add_scatter(x=x_hat, y=y_hat, mode='lines', name='Piecewise Linear Fit', line=dict(color='red', width=2))
fig.show()

/var/folders/28/cmh8kslx4f9gr_zckx5qp_sw0000gn/T/ipykernel_87397/3866217126.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/28/cmh8kslx4f9gr_zckx5qp_sw0000gn/T/ipykernel_87397/3866217126.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/nicolas/miniforge3/envs/analysis/lib/python3.12/site-packages/pwlf/pwlf.py:1108: RuntimeWarning:

invalid value encountered in scalar divide



In [9]:
# Preparing activation data

activated_all_time = activated_all_time_import.copy()
activated_all_time.Datum = pd.to_datetime(activated_all_time.Datum, format = "%d.%m.%Y")

#Only 01-11 June 2024
activated_june_2024 = activated_all_time[(activated_all_time.Datum.dt.month == 6) & (activated_all_time.Datum.dt.year == 2024) & (activated_all_time.Datum.dt.day <= 11)]

# Remove TSO specific columns, add product names, replace commas with dots, rename columns
activated_june_2024 = activated_june_2024.drop(columns = ['50Hertz (Positiv)','Amprion (Positiv)', 'TenneT TSO (Positiv)', 'TransnetBW (Positiv)','50Hertz (Negativ)', 'Amprion (Negativ)','TenneT TSO (Negativ)', 'TransnetBW (Negativ)'])
activated_june_2024['product_time'] = activated_june_2024.von.apply(lambda x: x.split(':')).apply(lambda x: int((4*int(x[0])) +(int(x[1])/15)+1)).apply(lambda x: (3-len(str(x)))*'0' + str(x))
activated_june_2024['Deutschland (Positiv)'] = activated_june_2024['Deutschland (Positiv)'].apply(lambda x: x.replace(',','.'))
activated_june_2024['Deutschland (Negativ)'] = activated_june_2024['Deutschland (Negativ)'].apply(lambda x: x.replace(',','.'))
activated_june_2024.rename(columns = {'Deutschland (Positiv)':'POS', 'Deutschland (Negativ)':'NEG'}, inplace = True)

#Separating Neg and Pos products into separate rows
activated_june_2024['time_prod_NEG'] = 'NEG_' + activated_june_2024['product_time'] 
activated_june_2024['time_prod_POS'] = 'POS_' + activated_june_2024['product_time']
NEG = activated_june_2024.rename(columns={'time_prod_NEG':'prod'}).set_index(['Datum','prod']).drop(columns = ['POS']).rename(columns={'NEG':'ACTIVATED'}).drop(columns=['time_prod_POS'])
POS = activated_june_2024.rename(columns={'time_prod_POS':'prod'}).set_index(['Datum','prod']).drop(columns = ['NEG']).rename(columns={'POS':'ACTIVATED'}).drop(columns=['time_prod_NEG'])
combined_activated = pd.concat([NEG,POS])

In [4]:
# Preparing bidding data

bids_june_2024 = bids_june_2024_import.copy()
bids_june_2024['DELIVERY_DATE'] = pd.to_datetime(bids_june_2024.DELIVERY_DATE,format = "%Y-%m-%d")

# Translate payment direction into sign for price
bids_june_2024['ENERGY_PRICE_[EUR/MWh]'] = bids_june_2024['ENERGY_PRICE_[EUR/MWh]']*(((bids_june_2024['ENERGY_PRICE_PAYMENT_DIRECTION'] == 'PROVIDER_TO_GRID') * -1) + ((bids_june_2024['ENERGY_PRICE_PAYMENT_DIRECTION'] == 'GRID_TO_PROVIDER') * 1))

# Sort by Dalivery_date, product then energy price (check order)
bids_sorted = bids_june_2024.sort_values(['DELIVERY_DATE','PRODUCT','ENERGY_PRICE_[EUR/MWh]'])

# Cumulative sums of offered capacity
bids_sorted['sum_cap'] = bids_sorted.groupby(['DELIVERY_DATE','PRODUCT'])['OFFERED_CAPACITY_[MW]'].cumsum()

bids_activated = bids_sorted.set_index(['DELIVERY_DATE','PRODUCT'])


In [19]:
bids_activated.index

MultiIndex([('2024-06-01', 'NEG_001'),
            ('2024-06-01', 'NEG_001'),
            ('2024-06-01', 'NEG_001'),
            ('2024-06-01', 'NEG_001'),
            ('2024-06-01', 'NEG_001'),
            ('2024-06-01', 'NEG_001'),
            ('2024-06-01', 'NEG_001'),
            ('2024-06-01', 'NEG_001'),
            ('2024-06-01', 'NEG_001'),
            ('2024-06-01', 'NEG_001'),
            ...
            ('2024-06-11', 'POS_021'),
            ('2024-06-11', 'POS_021'),
            ('2024-06-11', 'POS_021'),
            ('2024-06-11', 'POS_021'),
            ('2024-06-11', 'POS_021'),
            ('2024-06-11', 'POS_021'),
            ('2024-06-11', 'POS_021'),
            ('2024-06-11', 'POS_021'),
            ('2024-06-11', 'POS_021'),
            ('2024-06-11', 'POS_021')],
           names=['DELIVERY_DATE', 'PRODUCT'], length=1048575)

In [34]:
import plotly.express as px

first_D_pos_001 = bids_activated.loc['2024-06-01','POS_001']
first_D_pos_001['sum_cap'] = first_D_pos_001['sum_cap']/4
first_D_pos_001['avg_price'] = (first_D_pos_001['ENERGY_PRICE_[EUR/MWh]']*first_D_pos_001['OFFERED_CAPACITY_[MW]']/4).cumsum()/first_D_pos_001['sum_cap']

# Plotting the merit order using Plotly
fig = px.line(first_D_pos_001, x='sum_cap', y='ENERGY_PRICE_[EUR/MWh]', markers=True, title='Merit Order for POS_001 on 2024-06-01')
fig.update_layout(xaxis_title='Cumulative Capacity [MW]', yaxis_title='Energy Price [EUR/MWh]')
fig.add_scatter(x=first_D_pos_001['sum_cap'], y=first_D_pos_001['avg_price'], mode='lines', name='Average Price', line=dict(color='black', width=2))
fig.show()


/var/folders/28/cmh8kslx4f9gr_zckx5qp_sw0000gn/T/ipykernel_87397/2623539232.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/28/cmh8kslx4f9gr_zckx5qp_sw0000gn/T/ipykernel_87397/2623539232.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [33]:
first_D_pos_001 = bids_activated.loc['2024-06-01','POS_001']
first_D_pos_001['sum_cap'] = first_D_pos_001['sum_cap']/4
first_D_pos_001['avg_price'] = first_D_pos_001['ENERGY_PRICE_[EUR/MWh]'].cumsum()/first_D_pos_001['sum_cap']
first_D_pos_001

/var/folders/28/cmh8kslx4f9gr_zckx5qp_sw0000gn/T/ipykernel_87397/2400444081.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/28/cmh8kslx4f9gr_zckx5qp_sw0000gn/T/ipykernel_87397/2400444081.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,TYPE_OF_RESERVES,ENERGY_PRICE_[EUR/MWh],ENERGY_PRICE_PAYMENT_DIRECTION,OFFERED_CAPACITY_[MW],ALLOCATED_CAPACITY_[MW],COUNTRY,NOTE,sum_cap,avg_price
DELIVERY_DATE,,,,,,,,,
2024-06-01,aFRR,114.78,GRID_TO_PROVIDER,5,5,DE,NaN,1.25,91.824000
2024-06-01,aFRR,119.82,GRID_TO_PROVIDER,5,5,DE,NaN,2.50,93.840000
2024-06-01,aFRR,124.86,GRID_TO_PROVIDER,5,5,DE,NaN,3.75,95.856000
2024-06-01,aFRR,125.86,GRID_TO_PROVIDER,5,5,DE,NaN,5.00,97.064000
2024-06-01,aFRR,131.86,GRID_TO_PROVIDER,5,5,DE,NaN,6.25,98.748800
...,...,...,...,...,...,...,...,...,...
2024-06-01,aFRR,14999.00,GRID_TO_PROVIDER,20,20,DE,NaN,506.25,716.811240
2024-06-01,aFRR,14999.00,GRID_TO_PROVIDER,3,3,DE,NaN,507.00,745.334694
2024-06-01,aFRR,15000.00,GRID_TO_PROVIDER,3,3,DE,NaN,507.75,773.775854


In [52]:
# Aligning activation and bidding data

# Join both
bids_activated['act'] = combined_activated['ACTIVATED']

# Keep only activated energy bids (exclude partially activated)
bids_activated = bids_activated[bids_activated['sum_cap'] < bids_activated['act'].astype(float)]

#bids_activated[(bids_activated.index.get_level_values('DELIVERY_DATE') == pd.to_datetime('2024-06-05')) & (bids_activated.index.get_level_values('PRODUCT') == 'POS_048')]# == [pd.to_datetime('2024-06-01'), 'NEG_001']]
# Daily activation cost (assuming full activation for 15min) 
#((bids_activated['ENERGY_PRICE_[EUR/MWh]'].groupby('DELIVERY_DATE').sum())/4).astype(int)
((bids_activated['ENERGY_PRICE_[EUR/MWh]'].groupby('PRODUCT').sum())/4).astype(int)

PRODUCT
NEG_001     -353
NEG_002     -310
NEG_003     -250
NEG_004     -361
NEG_005     -492
           ...  
POS_092      971
POS_093    11843
POS_094     5552
POS_095     4790
POS_096     2428
Name: ENERGY_PRICE_[EUR/MWh], Length: 192, dtype: int64